In [1]:
from cl_v2 import trader
from cl_v2 import back_klines
from cl_v2 import cl
from cl_v2 import kcharts
from cl_v2.my import strategy_demo
import time
import datetime
import threading

In [2]:
# 量化配置
start_datetime = '2021-10-01 00:00:00'
end_datetime = '2021-12-01 00:00:00'
market = 'currency'
codes = ['BTC/USDT']
frequencys = ['30m', '5m']

STR_Class = strategy_demo.Strategy_Demo()

In [3]:
traders = {}
bks = {}
_start_datetime = datetime.datetime.strptime(start_datetime, '%Y-%m-%d %H:%M:%S')
_end_datetime = datetime.datetime.strptime(end_datetime, '%Y-%m-%d %H:%M:%S')

for code in codes:
    bks[code] = back_klines.BackKlines(market, code, _start_datetime, _end_datetime, frequencys)
    bks[code].start()
    traders[code] = trader.Trader(code, is_stock=False, is_futures=True, log=print, is_save_kline=True)
    traders[code].pos_space = 5
    traders[code].set_strategy(STR_Class)
    
    
def run(bk:back_klines.BackKlines, td:trader.Trader):
    start = time.time()
    while True:
        ok = bk.next(frequencys[-1])
        if ok is False:
            break
        klines = {}
        for f in frequencys:
            klines[f] = bk.show_klines[f]

        td.run(bk.code, klines)
    td.end()
    end = time.time()
    print('%s Total Run: %.4f s' % (td.name, end - start))

# 多线程运行程序
threads = {}
for code in codes:
    threads[code] = threading.Thread(target=run, args=(bks[code], traders[code]))
    threads[code].setDaemon(True)
    threads[code].start()
    

Code BTC/USDT F 30m len 5329
Code BTC/USDT F 5m len 19009
[BTC/USDT - 2021-10-02 00:35:00] // 3buy 做多买入（47508.81 - 4.126），原因： 高级别出现 3buy (TD: True Done: True)
[BTC/USDT - 2021-10-02 08:10:00] // 3buy 平仓做多（47698.19 - 4.126） 盈亏：781.3818800000299  (0.40%)，原因： 高级别笔（TD: True Done: True），多仓清仓
[BTC/USDT - 2021-10-05 08:15:00] // 3buy 做多买入（49417.24 - 3.9697），原因： 高级别出现 3buy (TD: True Done: True)
[BTC/USDT - 2021-10-05 12:50:00] // 3buy 平仓做多（49069.01 - 3.9697） 盈亏：-1382.3686309999612  (-0.70%)，原因： 高级别笔（TD: True Done: True），多仓清仓
[BTC/USDT - 2021-10-11 08:35:00] // 3buy 做多买入（54968.89 - 3.5639），原因： 高级别出现 3buy (TD: True Done: True)
[BTC/USDT - 2021-10-11 13:35:00] // 3buy 平仓做多（56227.09 - 3.5639） 盈亏：4484.098979999981  (2.29%)，原因： 高级别笔（TD: True Done: True），多仓清仓
[BTC/USDT - 2021-10-12 17:50:00] // 2sell 做空卖出（57083.1 - 3.4473），原因： 高级别出现 2sell (TD: True Done: True)
[BTC/USDT - 2021-10-13 05:45:00] // 2sell 平仓做空（56050.35 - 3.4473） 盈亏：3560.1990749999823 (1.81%)，原因： 高级别笔（TD: True Done: True），多仓清仓
[BTC/USDT -

In [4]:
# 打印现在执行到的时间情况
for code in codes:
    print('%s run datetime : %s' % (code, bks[code].now_date))

BTC/USDT run datetime : 2021-10-17 17:10:00


In [5]:
trader.traders_result(traders.values())

+------------+------+------+--------+---------+---------+---------+----------+----------+--------+
|   买卖点   | 成功 | 失败 |  胜率  |   盈利  |   亏损  |  净利润 | 平均盈利 | 平均亏损 | 盈亏比 |
+------------+------+------+--------+---------+---------+---------+----------+----------+--------+
|  一类买点  |  0   |  0   |   0%   |    0    |    0    |    0    |    0     |    0     |   0    |
|  二类买点  |  0   |  0   |   0%   |    0    |    0    |    0    |    0     |    0     |   0    |
| 类二类买点 |  0   |  0   |   0%   |    0    |    0    |    0    |    0     |    0     |   0    |
|  三类买点  |  2   |  2   | 50.0%  | 5265.48 | 3944.76 | 1320.72 | 2632.74  | 1972.38  | 1.3348 |
| 类三类买点 |  0   |  0   |   0%   |    0    |    0    |    0    |    0     |    0     |   0    |
|  一类卖点  |  1   |  0   | 100.0% |  691.94 |    0    |  691.94 |  691.94  |    0     |   0    |
|  二类卖点  |  1   |  0   | 100.0% |  3560.2 |    0    |  3560.2 |  3560.2  |    0     |   0    |
| 类二类卖点 |  1   |  0   | 100.0% | 3160.91 |    0    | 3160.91 | 3160

In [ ]:
renders = {}
for code in traders.keys():
    td = traders[code]
    renders[td.name] = {}
    for f in frequencys:
        renders[td.name][f] = td.show_kline(td.name, show_frequency=f)

In [ ]:
renders[codes[0]]['30m']

In [6]:
print('交易对,买卖点,开仓时间,平仓时间,类型,价格,数量,止损,盈亏,最大盈利,最大亏损,开仓信息,平仓信息,高级别买卖点,高级别停顿,高级别同类中枢,低级别分解买卖点,低级别扩展买卖点,低级别停顿,低级别同类中枢')
for td in traders.values():
    if td.name not in td.positions_history:
        continue
    for p in td.positions_history[td.name]:
        print('%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s' % (
            td.name, p.mmd, p.open_datetime, p.close_datetime, p.type, p.price, p.amount, p.loss_price, 
            p.profit_rate, p.max_profit_rate, p.max_loss_rate, p.open_msg.replace(',', ''), p.close_msg.replace(',', '')))

交易对,买卖点,开仓时间,平仓时间,类型,价格,数量,止损,盈亏,最大盈利,最大亏损,开仓信息,平仓信息,高级别买卖点,高级别停顿,高级别同类中枢,低级别分解买卖点,低级别扩展买卖点,低级别停顿,低级别同类中枢
BTC/USDT,3buy,2021-10-02 00:35:00,2021-10-02 08:10:00,做多,47508.81,4.126,46280.0,0.4,1.9811,-0.0939,高级别出现 3buy (TD: True Done: True),高级别笔（TD: True Done: True），多仓清仓
BTC/USDT,3buy,2021-10-05 08:15:00,2021-10-05 12:50:00,做多,49417.24,3.9697,48521.36,-0.7,0.9526,-0.5408,高级别出现 3buy (TD: True Done: True),高级别笔（TD: True Done: True），多仓清仓
BTC/USDT,3buy,2021-10-11 08:35:00,2021-10-11 13:35:00,做多,54968.89,3.5639,54329.39,2.29,3.4431,0,高级别出现 3buy (TD: True Done: True),高级别笔（TD: True Done: True），多仓清仓
BTC/USDT,2sell,2021-10-12 17:50:00,2021-10-13 05:45:00,做空,57083.1,3.4473,56512.269,1.81,5.839,-0.3887,高级别出现 2sell (TD: True Done: True),高级别笔（TD: True Done: True），多仓清仓
BTC/USDT,l2sell,2021-10-13 08:00:00,2021-10-13 18:25:00,做空,55915.91,3.5317,56560.62,1.6,2.9413,-0.8721,高级别出现 l2sell (TD: True Done: True),高级别笔（TD: True Done: True），多仓清仓
BTC/USDT,1sell,2021-10-14 12:00:00,2021-10-14 19:10:00,做空,58078.59,3.

In [8]:
kcharts.render_cash(traders[codes[0]].balances_history[codes[0]])